In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime, timedelta, date
%matplotlib inline
plt.style.use('ggplot')

import nltk
import string

from nltk.corpus import stopwords
pstemmer = nltk.PorterStemmer()

In [5]:
df = pd.read_csv('data/winemag-data-130k-v2.csv')

In [41]:
df.sample(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
113468,113468,US,"One of the more substantial examples of rosé, ...",NaN,89,16.0,California,Dunnigan Hills,Central Valley,Jim Gordon,@gordone_cellars,Tinto Rey 2016 Rosé (Dunnigan Hills),Rosé,Tinto Rey
41286,41286,Italy,Playful aromas of sweet fruit and watermelon c...,NaN,86,12.0,Southern Italy,Taburno,NaN,NaN,NaN,Ocone 2009 Piedirosso (Taburno),Piedirosso,Ocone
4790,4790,US,"Shows plenty of oak influence, wrapping around...",White Elephant,85,17.0,California,California,California Other,NaN,NaN,Vinum 2010 White Elephant White (California),White Blend,Vinum
57928,57928,France,As rich and intense as the same producer's Non...,Bouzy Grand Cru Brut Rosé,90,45.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,P. Louis Martini NV Bouzy Grand Cru Brut Rosé ...,Champagne Blend,P. Louis Martini
16045,16045,Portugal,"A creamy textured, citrus-flavored wine that i...",Adega de Pegões Colheita Seleccionada,87,NaN,Península de Setúbal,NaN,NaN,Roger Voss,@vossroger,Cooperativa Agricola de Santo Isidro de Pegoes...,Portuguese White,Cooperativa Agricola de Santo Isidro de Pegoes


In [38]:
df.description.sample(20).values

array([ 'Pastry notes blend with ripe berry aromas and hints of vanilla and raisin. This is rubbery and tannic in feel, with flavors of cherry, plum, cassis and herbal spice. On the finish, an herbal flavor note is typical of Chilean Cabernet.',
       "A wisp of smoke mingles with blossom, apple and pear flavors in this lively, fresh Pinot Gris. It's quite tart with acidity, but pleasantly fruity with a subtle hint of minerality on the finish.",
       'Darkly colored, this wine opens with aromas of menthol, espresso and plum. The one-dimensional palate is surprisingly dense for the vintage, offering black cherry, espresso and clove alongside polished tannins.',
       'Elegantly smooth in the mouth, like silk and velvet—this has rich, forward blackberry, black currant and oak flavors. It feels dramatic and complex, a white-tablecloth Cabernet for pairing with fine meats and cheeses. Seems best now and for a couple more years.',
       'Made in a deliciously oaky style, this smells li

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
Unnamed: 0               129971 non-null int64
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 13.9+ MB


In [98]:
for col in df.columns:
    print(col, df[col].unique().size)

Unnamed: 0 129971
country 44
description 119955
designation 37980
points 21
price 391
province 426
region_1 1230
region_2 18
taster_name 20
taster_twitter_handle 16
title 118840
variety 708
winery 16757


In [12]:
df['taster_name'].fillna('none').value_counts(1)

none                  0.201922
Roger Voss            0.196305
Michael Schachner     0.116441
Kerin O’Keefe         0.082911
Virginie Boone        0.073378
Paul Gregutt          0.073339
Matt Kettmann         0.048719
Joe Czerwinski        0.039601
Sean P. Sullivan      0.038209
Anna Lee C. Iijima    0.033969
Jim Gordon            0.032138
Anne Krebiehl MW      0.028352
Lauren Buzzeo         0.014119
Susan Kostrzewa       0.008348
Mike DeSimone         0.003955
Jeff Jenssen          0.003778
Alexander Peartree    0.003193
Carrie Dykes          0.001069
Fiona Adams           0.000208
Christina Pickard     0.000046
Name: taster_name, dtype: float64

In [42]:
df_nona = df[df['taster_name'].fillna('N') != 'N']

In [43]:
label_map = {val: idx for idx, val in enumerate(df['taster_name'].unique())}

In [33]:
y = df_nona['taster_name'].replace(label_map).values

In [34]:
reviews = df_nona.description

In [35]:
swords = set(stopwords.words('english'))

In [36]:
ps = PorterStemmer()

In [44]:
word_vecs = reviews\
    .apply(lambda x: [re.sub("[\W]", '', i).lower().strip() for i in x.split()])\
    .apply(lambda x: [ps.stem(i) for i in x if i not in swords and len(i) > 3])

In [45]:
word_vecs.sample(10)

37673     [perfum, wine, thin, textur, pure, applejuic, ...
81121     [pinch, smell, first, note, turpentin, air, el...
67456     [dark, berri, press, flower, iron, leather, sp...
12015     [three, stand, wine, aviz, ambonnay, grand, vi...
24789     [dens, nose, black, fruit, aroma, mild, herb, ...
57371     [wine, rich, bring, togeth, great, white, grap...
110924    [wine, sanglièr, rang, delici, fine, balanc, f...
45111     [blend, cabernet, franc, syrah, merlot, cabern...
4873      [underbrush, dark, berri, press, violet, aroma...
15534     [rather, delic, viognier, still, lightli, yeas...
Name: description, dtype: object

In [46]:
back2sent = word_vecs.apply(lambda x: ' '.join(x))

In [47]:
transform = TfidfVectorizer(lowercase=False, min_df=.05)

In [53]:
tf_idf_matrix = transform.fit_transform(back2sent.values)

In [54]:
from sklearn.naive_bayes import GaussianNB

In [60]:
test_train = np.random.random(word_vecs.shape[0]) < .8
Xtr = tf_idf_matrix[test_train]
Xte = tf_idf_matrix[~test_train]
ytr = y[test_train]
yte = y[~test_train]

In [61]:
model = GaussianNB()

In [64]:
model.fit(Xtr.toarray(), ytr)

GaussianNB(priors=None)

In [67]:
model.predict_proba(Xte.toarray()).shape

(20894, 19)

In [68]:
model.score(Xte.toarray(), yte)

0.36905331674164832

In [69]:
from sklearn.ensemble import RandomForestClassifier

In [84]:
model2 = RandomForestClassifier(n_estimators=250, n_jobs=-1)
model2.fit(Xtr.toarray(), ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=250, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [85]:
model2.score(Xte.toarray(), yte)

0.76084043266009382

In [87]:
## run this line if needed xgboost is not a part of Sklearn
## we could also use GradientBoostingClassifier, but since it does not support 
## parallel processing, and XGboost typically gives near the same performance
## we will use XGboost


!pip install xgboost

    100% |████████████████████████████████| 1.1MB 13.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for xgboost ... done
  Stored in directory: /Users/travis/Library/Caches/pip/wheels/0a/78/34/a11088de607478e7447698587aea75e3e20ecdddd1dd10d751
Successfully built xgboost
pyspark 2.2.1 requires py4j==0.10.4, which is not installed.


In [88]:
import xgboost as xgb

In [89]:
modelxg3 = xgb.XGBClassifier(max_depth=3, learning_rate=.05, n_estimators=250, n_jobs=-1)

In [91]:
modelxg3.fit(Xtr.toarray(), ytr)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=-1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [92]:
modelxg3.score(Xte.toarray(), yte)

0.73073609648702975

In [93]:
modelxg5 = xgb.XGBClassifier(max_depth=5, learning_rate=.05, n_estimators=250, n_jobs=-1)
modelxg5.fit(Xtr.toarray(), ytr)
modelxg5.score(Xte.toarray(), yte)

0.75959605628410065

In [94]:
modelxg_lr1 = xgb.XGBClassifier(max_depth=5, learning_rate=.1, n_estimators=250, n_jobs=-1)
modelxg_lr1.fit(Xtr.toarray(), ytr)
modelxg_lr1.score(Xte.toarray(), yte)

0.77964966018952808

In [79]:
# dense_matrix = pd.DataFrame(tf_idf_matrix.todense())

# ## these may be in order of count, not the orde of the columns so this is were i messed up
# dense_matrix.columns = [v for v in transform.vocabulary_.keys()]

# dense_matrix.columns

# dense_matrix.sample(10)

# country_dm = dense_matrix.copy()

# country_dm['y'] = df['country']

In [ ]:
start = datetime.now()
#process things
print((datetime.now() - start).seconds())

In [ ]:
%%timeit -n 1